In [1]:
import glob, re, os
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

import sys

In [2]:
class MyCallback(CallbackAny2Vec):
    '''
    This callback performs the evaluation of the word embeddings through a series of tests.
    The tests are described in the paper https://arxiv.org/abs/1507.05523
    The tests implemented are "ws", "tfl", and "sem&syn"
    '''
    
    def __init__(self, embedding_name):
        self.epoch = 0
        self.embedding_name = embedding_name
        pass

    
    def on_epoch_end(self, model):
        
        if self.epoch%5==0:
            print("Epoch #{} end".format(self.epoch))
        
        # eval every 5 epochs and for the last one
        if self.epoch%5==0 or self.epoch==model.epochs-1:
            #training_loss = model.get_latest_training_loss()
            #print('Training loss: ', training_loss)

            # save the embeddings
            model.wv.save_word2vec_format('word_vectors.txt', write_header=False)
            
            #interpreter = "/Users/lorenzo/miniconda3/envs/env_lyrics/bin/python"
            interpreter = sys.executable
            
            # run the evaluations
            cmd = f'{interpreter} ../src/run_evaluation_embeddings.py \
            --emb_dim {model.vector_size} \
            --eval_files_path ../data/evaluation_tests_word_embedding/ \
            --embedding_file word_vectors.txt \
            --embedding_name {self.embedding_name+f"_epoch-{self.epoch}"} \
            --results_dir ../results/experiment_W2V_several_runs_WEAT_2/Word_embedding_quality_W2V/ \
            --log_file ../logs/experiment_W2V_several_runs_WEAT_2/Word_embedding_quality_W2V/log_{self.embedding_name+f"_epoch-{self.epoch}"}.log \
            --upper_and_lower True'
            os.system(cmd)
            
            # delete word embedding file
            os.remove('word_vectors.txt')
            
        self.epoch += 1

In [3]:
dim = 300
w = 5
n_epochs = 40

n_iterations = 5

# keep all corpus files
corpus_files = glob.glob('../data/corpora_lemmatized/*')
print("Corpus files: ", corpus_files)
print()

for n in range(n_iterations):
    for corpus_file in corpus_files:

        corpus_name = corpus_file.split("/")[-1].rstrip(".cor")
        corpus_type = corpus_file.split("/")[-2]

        embedding_name = f'W2V_lemmatized_iter-{n}_dim-{dim}_w-{w}_alpha-default_epochs-{n_epochs}_corpus-{corpus_name}' 

        print("Start training corpus: ", corpus_name)
        print("Embedding name: ", embedding_name)

        callback = MyCallback(embedding_name=embedding_name)
        model = Word2Vec(vector_size=dim, 
                         window=w, 
                         sg=1,
                         hs=0,
                         negative=5,  # default
                         ns_exponent=0.75, # default
                         #alpha=0.1, # Use only if making multiple calls to train(), when you want to manage the alpha learning-rate yourself (not recommended).
                         #min_alpha=0.0001, # default, linear decay of lr
                         min_count=5,
                         max_vocab_size=100000, 
                         workers=3,
                         )


        model.build_vocab(corpus_file=corpus_file)
        print('Vocab done...')
        print('Corpus count: ', model.corpus_count)
        print('Corpus total words: ', model.corpus_total_words)

        model.train(corpus_file=corpus_file,  
                    callbacks=[callback], 
                    total_examples=model.corpus_count,
                    total_words=model.corpus_total_words,
                    epochs=n_epochs
                    )
        # save the model
        model.wv.save_word2vec_format("../results/experiment_W2V_several_runs_WEAT_2/Word_vectors/"+embedding_name+'.txt', 
                                      write_header=False)


        print(f'End training of {embedding_name}')
        print()



Corpus files:  ['../data/corpora_lemmatized/female_person_artist_lyrics.cor', '../data/corpora_lemmatized/male_person_artist_lyrics.cor', '../data/corpora_lemmatized/all_person_artist_lyrics.cor']

Start training corpus:  female_person_artist_lyrics
Embedding name:  W2V_lemmatized_iter-0_dim-300_w-5_alpha-default_epochs-40_corpus-female_person_artist_lyrics
Vocab done...
Corpus count:  70880
Corpus total words:  15949187
Epoch #0 end


[2021-12-24 17:14:51,259 INFO] Start computation quality word embedding tests. Results will be saved at ../results/experiment_W2V_several_runs_WEAT_2/Word_embedding_quality_W2V/
[2021-12-24 17:14:51,261 INFO] Number of unique words for tests: 2066
[2021-12-24 17:14:51,262 INFO] Reading word vectors...
[2021-12-24 17:14:51,643 INFO] Number of word vectors read: 10000...
[2021-12-24 17:14:52,020 INFO] Number of word vectors read: 20000...
[2021-12-24 17:14:52,121 INFO] Number of word vectors found: 1099
[2021-12-24 17:14:52,121 INFO] Running: sed -i '' '1s/^/1099 300\'$'\n/' ../data/evaluation_tests_word_embedding/temp_embs.txt
[2021-12-24 17:14:52,144 INFO] Running WS test...
[2021-12-24 17:14:52,144 INFO] Calling command: ./../data/evaluation_tests_word_embedding/ws/ws ../data/evaluation_tests_word_embedding/temp_embs.txt > ../results/experiment_W2V_several_runs_WEAT_2/Word_embedding_quality_W2V/W2V_lemmatized_iter-0_dim-300_w-5_alpha-default_epochs-40_corpus-female_person_artist_lyric

Epoch #5 end


[2021-12-24 17:18:15,732 INFO] Start computation quality word embedding tests. Results will be saved at ../results/experiment_W2V_several_runs_WEAT_2/Word_embedding_quality_W2V/
[2021-12-24 17:18:15,739 INFO] Number of unique words for tests: 2066
[2021-12-24 17:18:15,739 INFO] Reading word vectors...
[2021-12-24 17:18:16,146 INFO] Number of word vectors read: 10000...
[2021-12-24 17:18:16,532 INFO] Number of word vectors read: 20000...
[2021-12-24 17:18:16,634 INFO] Number of word vectors found: 1099
[2021-12-24 17:18:16,634 INFO] Running: sed -i '' '1s/^/1099 300\'$'\n/' ../data/evaluation_tests_word_embedding/temp_embs.txt
[2021-12-24 17:18:16,655 INFO] Running WS test...
[2021-12-24 17:18:16,655 INFO] Calling command: ./../data/evaluation_tests_word_embedding/ws/ws ../data/evaluation_tests_word_embedding/temp_embs.txt > ../results/experiment_W2V_several_runs_WEAT_2/Word_embedding_quality_W2V/W2V_lemmatized_iter-0_dim-300_w-5_alpha-default_epochs-40_corpus-female_person_artist_lyric

Epoch #10 end


...

Epoch #30 end


[2021-12-25 12:28:10,118 INFO] Start computation quality word embedding tests. Results will be saved at ../results/experiment_W2V_several_runs_WEAT_2/Word_embedding_quality_W2V/
[2021-12-25 12:28:10,120 INFO] Number of unique words for tests: 2066
[2021-12-25 12:28:10,120 INFO] Reading word vectors...
[2021-12-25 12:28:10,721 INFO] Number of word vectors read: 10000...
[2021-12-25 12:28:11,310 INFO] Number of word vectors read: 20000...
[2021-12-25 12:28:11,873 INFO] Number of word vectors read: 30000...
[2021-12-25 12:28:12,437 INFO] Number of word vectors read: 40000...
[2021-12-25 12:28:13,056 INFO] Number of word vectors found: 1341
[2021-12-25 12:28:13,057 INFO] Running: sed -i '' '1s/^/1341 300\'$'\n/' ../data/evaluation_tests_word_embedding/temp_embs.txt
[2021-12-25 12:28:13,091 INFO] Running WS test...
[2021-12-25 12:28:13,093 INFO] Calling command: ./../data/evaluation_tests_word_embedding/ws/ws ../data/evaluation_tests_word_embedding/temp_embs.txt > ../results/experiment_W2V_

Epoch #35 end


[2021-12-25 12:43:50,033 INFO] Start computation quality word embedding tests. Results will be saved at ../results/experiment_W2V_several_runs_WEAT_2/Word_embedding_quality_W2V/
[2021-12-25 12:43:50,035 INFO] Number of unique words for tests: 2066
[2021-12-25 12:43:50,036 INFO] Reading word vectors...
[2021-12-25 12:43:50,444 INFO] Number of word vectors read: 10000...
[2021-12-25 12:43:51,131 INFO] Number of word vectors read: 20000...
[2021-12-25 12:43:52,674 INFO] Number of word vectors read: 30000...
[2021-12-25 12:43:53,586 INFO] Number of word vectors read: 40000...
[2021-12-25 12:43:53,996 INFO] Number of word vectors found: 1341
[2021-12-25 12:43:53,996 INFO] Running: sed -i '' '1s/^/1341 300\'$'\n/' ../data/evaluation_tests_word_embedding/temp_embs.txt
[2021-12-25 12:43:54,028 INFO] Running WS test...
[2021-12-25 12:43:54,029 INFO] Calling command: ./../data/evaluation_tests_word_embedding/ws/ws ../data/evaluation_tests_word_embedding/temp_embs.txt > ../results/experiment_W2V_

End training of W2V_lemmatized_iter-4_dim-300_w-5_alpha-default_epochs-40_corpus-all_person_artist_lyrics

